# PREDICTING AIR QUALITY IN NAIROBI.

### Importing Labraries

In [2]:
import pandas as pd

from pprint import PrettyPrinter
from pymongo import MongoClient

Instantiate a `PrettyPrinter`, and assign it to the variable `pp`.

In [3]:
pp = PrettyPrinter(indent=2)

Create a client that connects to the database running at `localhost` on port `27017`

In [4]:
client = MongoClient(host="localhost", port=27017)

In [6]:
#list(client.list_databases())
pp.pprint(list(client.list_databases()))

[ {'empty': False, 'name': 'admin', 'sizeOnDisk': 8515584},
  {'empty': False, 'name': 'config', 'sizeOnDisk': 73728},
  {'empty': False, 'name': 'local', 'sizeOnDisk': 73728}]


Assign the `"air-quality"` database to the variable `db`.

In [5]:
db = client["air_quality"]

In [6]:
#list(db.list_collections())[0]
for c in db.list_collections():
    print(c["name"])

nairobi
air_quality_in_abuja


Assign the `"nairobi"` collection in `db` to the variable name `nairobi`.

In [9]:
nairobi = db["nairobi"]

Use the `count_documents`  method to see how many documents are in the `nairobi` collection.

In [10]:
nairobi.count_documents({})

667776

Use the`find_one` method to retrieve one document from the `nairobi` collection, and assign it to the variable name `result`.

In [11]:
result = nairobi.find_one({})
pp.pprint(result)

{ '_id': ObjectId('6486cd360b239e8cecce06b8'),
  'lat': -1.289,
  'location': 3573,
  'lon': 36.825,
  'sensor_id': 50,
  'sensor_type': 'DHT22',
  'timestamp': datetime.datetime(2023, 5, 1, 0, 0, 0, 555000),
  'value': 99.9,
  'value_type': 'humidity'}


In [12]:
nairobi.distinct("location")

[7, 33, 3573, 3576, 3579, 3580, 3596]

In [13]:
nairobi.count_documents({"location": 7})

66982

In [42]:
nairobi.count_documents({"location": 33})

68483

In [43]:
nairobi.count_documents({"location": 3573})

361708

In [44]:
nairobi.count_documents({"location": 3576})

161070

In [45]:
nairobi.count_documents({"location": 3579})

1875

In [46]:
nairobi.count_documents({"location": 3580})

3355

In [40]:
nairobi.count_documents({"location": 3596})

4303

Use the `aggregate` method to determine how many readings there are for each site in the `nairobi` collection.

In [14]:
result = nairobi.aggregate(
    [
        {
            "$group": {"_id": "$location", "count": {"$count": {}}}
        }
    ]
)
pp.pprint(list(result))

[ {'_id': 3580, 'count': 3355},
  {'_id': 3576, 'count': 161070},
  {'_id': 3596, 'count': 4303},
  {'_id': 3573, 'count': 361708},
  {'_id': 33, 'count': 68483},
  {'_id': 7, 'count': 66982},
  {'_id': 3579, 'count': 1875}]


In [15]:
nairobi.distinct("value_type")

['P0', 'P1', 'P2', 'humidity', 'temperature']

In [16]:
result = nairobi.find({"value_type": "P2"}).limit(3)
pp.pprint(list(result))

[ { '_id': ObjectId('6486cd360b239e8cecce06ba'),
    'lat': -1.289,
    'location': 3573,
    'lon': 36.825,
    'sensor_id': 49,
    'sensor_type': 'pms5003',
    'timestamp': datetime.datetime(2023, 5, 1, 0, 0, 34, 240000),
    'value': 39.0,
    'value_type': 'P2'},
  { '_id': ObjectId('6486cd360b239e8cecce06bf'),
    'lat': -1.298,
    'location': 7,
    'lon': 36.791,
    'sensor_id': 103,
    'sensor_type': 'SDS011',
    'timestamp': datetime.datetime(2023, 5, 1, 0, 0, 45, 724000),
    'value': 1,
    'value_type': 'P2'},
  { '_id': ObjectId('6486cd360b239e8cecce06c3'),
    'lat': -1.289,
    'location': 3573,
    'lon': 36.825,
    'sensor_id': 49,
    'sensor_type': 'pms5003',
    'timestamp': datetime.datetime(2023, 5, 1, 0, 1, 7, 58000),
    'value': 52.25,
    'value_type': 'P2'}]


Use the `aggregate` method to calculate how many readings there are for each type (`"humidity"`, `"temperature"`, `"P2"`, and `"P1"`) in site `7`, and these step will be repeated on the remaining location.

In [17]:
result = nairobi.aggregate(
    [
        {"$match": {"location": 7}},
        {
            "$group": {"_id": "$value_type", "count": {"$count": {}}}
        }
    ]
)
pp.pprint(list(result))

[ {'_id': 'P2', 'count': 16745},
  {'_id': 'P1', 'count': 16745},
  {'_id': 'temperature', 'count': 16746},
  {'_id': 'humidity', 'count': 16746}]


In [18]:
result = nairobi.aggregate(
    [
        {
            "$match": {"location": 33}
        },
        {
            "$group": {"_id": "$value_type", "count": {"$count": {}}}
        }
    ]
)
pp.pprint(list(result))

[ {'_id': 'humidity', 'count': 13384},
  {'_id': 'P0', 'count': 13905},
  {'_id': 'P1', 'count': 13905},
  {'_id': 'temperature', 'count': 13384},
  {'_id': 'P2', 'count': 13905}]


In [19]:
result = nairobi.aggregate(
    [
        {
            "$match": {"location": 3573}
        },
        {
            "$group": {"_id": "$value_type", "count": {"$count": {}}}
        }
    ]
)
pp.pprint(list(result))

[ {'_id': 'temperature', 'count': 72350},
  {'_id': 'P1', 'count': 72336},
  {'_id': 'humidity', 'count': 72350},
  {'_id': 'P0', 'count': 72336},
  {'_id': 'P2', 'count': 72336}]


In [20]:
result = nairobi.aggregate(
    [
        {
            "$match": {"location": 3576}
        },
        {
            "$group": {"_id": "$value_type", "count": {"$count": {}}}
        }
    ]
)
pp.pprint(list(result))

[ {'_id': 'humidity', 'count': 32223},
  {'_id': 'P2', 'count': 32208},
  {'_id': 'P0', 'count': 32208},
  {'_id': 'P1', 'count': 32208},
  {'_id': 'temperature', 'count': 32223}]


In [21]:
result = nairobi.aggregate(
    [
        {
            "$match": {"location": 3579}
        },
        {
            "$group": {"_id": "$value_type", "count": {"$count": {}}}
        }
    ]
)
pp.pprint(list(result))

[ {'_id': 'P2', 'count': 375},
  {'_id': 'P0', 'count': 375},
  {'_id': 'temperature', 'count': 375},
  {'_id': 'humidity', 'count': 375},
  {'_id': 'P1', 'count': 375}]


In [22]:
result = nairobi.aggregate(
    [
        {
            "$match": {"location": 3580}
        },
        {
            "$group": {"_id": "$value_type", "count": {"$count": {}}}
        }
    ]
)
pp.pprint(list(result))

[ {'_id': 'humidity', 'count': 665},
  {'_id': 'P2', 'count': 675},
  {'_id': 'P0', 'count': 675},
  {'_id': 'P1', 'count': 675},
  {'_id': 'temperature', 'count': 665}]


In [23]:
result = nairobi.aggregate(
    [
        {
            "$match": {"location": 3596}
        },
        {
            "$group": {"_id": "$value_type", "count": {"$count": {}}}
        }
    ]
)
pp.pprint(list(result))

[ {'_id': 'humidity', 'count': 742},
  {'_id': 'P2', 'count': 1126},
  {'_id': 'P0', 'count': 567},
  {'_id': 'P1', 'count': 1126},
  {'_id': 'temperature', 'count': 742}]


Use the `find` method to retrieve the PM 2.5 readings from location `3573`. i will limit the results to 3 records only. Since i won't need the metadata for our model, use the `projection` argument to limit the results to the `"P2"` and `"timestamp"` keys only.

In [24]:
result = nairobi.find(
    {"value_type": "P2", "location": 3573},
    projection = {"P2": 1, "value": 1, "timestamp": 1, "_id": 0}
)
pp.pprint(result[0])

{'timestamp': datetime.datetime(2023, 5, 1, 0, 0, 34, 240000), 'value': 39.0}


Read records from your `result` into the DataFrame `df`. Be sure to set the index to `"timestamp"`.

In [25]:
df = pd.DataFrame(result).set_index("timestamp")
df.head()

,value
timestamp,
2023-05-01 00:00:34.240,39.00
2023-05-01 00:01:07.058,52.25
2023-05-01 00:01:39.115,58.00
2023-05-01 00:02:10.942,32.80
2023-05-01 00:02:42.789,42.00


In [27]:
df["P2"] = df["value"]
df.drop(columns=["value"])

,P2
timestamp,
2023-05-01 00:00:34.240,39.00
2023-05-01 00:01:07.058,52.25
2023-05-01 00:01:39.115,58.00
2023-05-01 00:02:10.942,32.80
2023-05-01 00:02:42.789,42.00
...,...
2023-05-31 23:55:08.388,31.00
2023-05-31 23:55:40.226,30.50
2023-05-31 23:56:11.864,25.20
